<a href="https://colab.research.google.com/github/acrossariver/my_gcola/blob/main/%E7%B5%B1%E5%90%88%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0(N225%2BP333%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 統合スクリーニング_N225とP333


In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import random
from io import BytesIO

# ==== N225用スクリーニング条件 ====
N225_CONDITIONS = {
    "max_per": 15,
    "max_pbr": 3.0,
    "min_dividend_yield": 3.00,
    "min_up_change_rate": 0.20,
    "min_down_change_rate": -0.10,
    "days": 31
}

# ==== P333用スクリーニング条件 ====
P333_CONDITIONS = {
    "max_per": 20,
    "max_pbr": 4.0,
    "min_dividend_yield": 4.00,
    "min_up_change_rate": 0.20,
    "min_down_change_rate": -0.20,
    "days": 31
}

PRIME_RANDOM = 333

# ==== スクリーニング関数 ====
def screen_stocks(tickers, category_label, name_map, conditions):
    up_list, down_list = [], []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            per = info.get("trailingPE")
            pbr = info.get("priceToBook")
            div_yield = info.get("dividendYield")

            if None in [per, pbr, div_yield]:
                continue

            if (
                per <= conditions["max_per"]
                and pbr <= conditions["max_pbr"]
                and div_yield >= conditions["min_dividend_yield"]
            ):
                end = datetime.datetime.today()
                start = end - datetime.timedelta(days=conditions["days"])
                hist = stock.history(start=start, end=end)

                if len(hist) < 2:
                    continue

                start_price = hist["Close"].iloc[0]
                end_price = hist["Close"].iloc[-1]
                change_rate = (end_price - start_price) / start_price

                result = {
                    "カテゴリー": category_label,
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{div_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",
                    "上昇/下降": "上昇" if change_rate >= conditions["min_up_change_rate"] else "下降"
                }

                if change_rate >= conditions["min_up_change_rate"]:
                    up_list.append(result)
                elif change_rate <= conditions["min_down_change_rate"]:
                    down_list.append(result)
        except Exception as e:
            print(f"エラー: {ticker} - {e}")

    # DataFrame化と銘柄名マージ
    df_up = pd.DataFrame(up_list)
    df_down = pd.DataFrame(down_list)

    if not df_up.empty:
        df_up = df_up.merge(name_map, on="銘柄コード", how="left")
        cols = ['カテゴリー', '銘柄コード', '銘柄名'] + [c for c in df_up.columns if c not in ['カテゴリー', '銘柄コード', '銘柄名']]
        df_up = df_up[cols]

    if not df_down.empty:
        df_down = df_down.merge(name_map, on="銘柄コード", how="left")
        cols = ['カテゴリー', '銘柄コード', '銘柄名'] + [c for c in df_down.columns if c not in ['カテゴリー', '銘柄コード', '銘柄名']]
        df_down = df_down[cols]

    return df_up, df_down


# ==== 日経225のティッカー取得 ====
def get_nikkei225():
    url = "https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv"
    df = pd.read_csv(url, encoding='shift_jis')
    df['銘柄コード'] = df['コード'].astype(str) + '.T'
    return df['銘柄コード'].tolist(), df[['銘柄コード', '銘柄名']]


# ==== 東証プライム銘柄をExcelから取得 ====
def get_prime333():
    xls_url = 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls'
    response = requests.get(xls_url)
    df = pd.read_excel(BytesIO(response.content), sheet_name=0)
    df.columns = df.columns.str.strip()

    # プライム市場の銘柄を抽出し、明示的にコピー
    df = df[df['市場・商品区分'].str.contains('プライム', na=False)].copy()

    # 銘柄コード整形
    df['銘柄コード'] = df['コード'].astype(str).str.zfill(4) + '.T'

    tickers = random.sample(df['銘柄コード'].tolist(), PRIME_RANDOM)
    return tickers, df[['銘柄コード', '銘柄名']]



# ==== 実行 ====
tickers_n225, name_map_n225 = get_nikkei225()
tickers_prime, name_map_prime = get_prime333()

print(f"▶ N225銘柄数: {len(tickers_n225)} / P333銘柄数: {len(tickers_prime)}")

# N225スクリーニング
df_up_n225, df_down_n225 = screen_stocks(
    tickers_n225, "N225", name_map_n225, N225_CONDITIONS
)

# P333スクリーニング
df_up_p333, df_down_p333 = screen_stocks(
    tickers_prime, "P333", name_map_prime, P333_CONDITIONS
)

# ==== 結合 ====
df_all_n225 = pd.concat([df_up_n225, df_down_n225], ignore_index=True)
df_all_p333 = pd.concat([df_up_p333, df_down_p333], ignore_index=True)
df_all_total = pd.concat([df_all_n225, df_all_p333], ignore_index=True)




# ==== 結果表示 ====
print(f"\n◎ 総合 結果件数: {len(df_all_total)}")
display(df_all_total)

from google.colab import files

# ==== スクリーニング日を1列目に追加 ====
today_str = datetime.datetime.today().strftime("%Y/%m/%d")
df_all_total.insert(0, "スクリーニング日", today_str)

# ==== CSV保存 & ダウンロード ====
df_all_total.to_csv("N225_P333.csv", index=False, encoding='utf-8-sig')
files.download("N225_P333.csv")



▶ N225銘柄数: 225 / P333銘柄数: 333

◎ 総合 結果件数: 3


,カテゴリー,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,N225,8015.T,豊田通商,8.576340,1.184283,3.74%,23.03%,上昇
1,N225,6724.T,エプソン,11.376430,0.764286,3.85%,-13.30%,下降
2,P333,2175.T,エス・エム・エス,18.691853,2.381016,4.30%,20.66%,上昇


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>